In [ ]:
%%time
import pandas as pd
import numpy as np
pan_gene_expre=pd.read_table('data/PanCancer/tcga_RSEM_gene_tpm',sep='\t',index_col=0).sort_index(axis='rows').sort_index(axis='columns')

In [ ]:
pan_gene_expre

In [ ]:
print('number of genes is {},number of samples is {}.'.format(*pan_gene_expre.shape))

In [ ]:
pan_gene_expre.isnull().values.any()#check if there is any NA factor in this dataframe

In [ ]:
pan_sample_types=pd.read_table('data/PanCancer/TCGA_phenotype_denseDataOnlyDownload.tsv',index_col=0).sort_index(axis='rows')

In [ ]:
pan_sample_types

In [ ]:
pan_sample_types.index.duplicated().any()

In [ ]:
pan_sample_types.index

In [ ]:
pan_gene_expre.index

In [ ]:
sample_coincident=pan_gene_expre.columns.intersection(pan_sample_types.index)#select samples in the pan_gene_expre that are coincident with pan_sample_types
sample_coincident

In [ ]:
pan_gene_expre.columns.difference(pan_sample_types.index)
#there is one sample that is not exist in the sample types table

In [ ]:
pan_sample_types.shape
pan_sample_types=pan_sample_types.loc[sample_coincident]
pan_sample_types.shape
#只留下了在两个数据集中都存在的样本的对应数据

In [ ]:
pan_sample_types.isnull().any()

In [ ]:
#get the counts of the sample types
pan_sample_types.sample_type.value_counts(normalize=False,ascending=True)

In [ ]:
pan_sample_types['pathology_status']=pan_sample_types.apply(lambda row: 'Normal' if row['sample_type']=='Solid Tissue Normal' else 'Tumor',axis=1)

In [ ]:
pan_sample_types.pathology_status.value_counts(normalize=False,ascending=True)

In [ ]:
pan_clinical=pd.read_table('data/PanCancer/Survival_SupplementalTable_S1_20171025_xena_sp',index_col=0).sort_index(axis='rows')

In [ ]:
pan_clinical

In [ ]:
#check if there are same IDs in clinical data
pan_clinical.index.duplicated().any()

In [ ]:
#we select the intersection of the fore-mentioned three datasets
gene_types_clinical_coincident=pan_gene_expre.columns.intersection(pan_clinical.index)

In [ ]:
len(gene_types_clinical_coincident)

In [ ]:
pan_clinical.index

In [ ]:
pan_clinical=pan_clinical.loc[gene_types_clinical_coincident]

In [ ]:
pan_clinical.shape

In [ ]:
variable='OS'
print('number of samples with Overall Survival:',sum(pan_clinical[variable].value_counts(normalize=False)))
pan_clinical[variable].value_counts(normalize=True)

In [ ]:
variable='PFI'
print('number of samples with Progression-free interval:',sum(pan_clinical[variable].value_counts(normalize=False)))
pan_clinical[variable].value_counts(normalize=True)

In [ ]:
variable='DSS'
print('number of samples with Disease-specific Survival:',sum(pan_clinical[variable].value_counts(normalize=False)))
pan_clinical[variable].value_counts(normalize=True)

In [ ]:
%%time
with pd.HDFStore('data/Preprocessed_Data/PanCancer.h5','w') as store:
    store['gene_expression']=pan_gene_expre.transpose()
    store['sample_type']=pan_sample_types
    store['clinical']=pan_clinical